## Import Stuff


In [1]:
from pathlib import Path
import dotenv
import kagglehub
import numpy as np
import pandas as pd

## Load Environment Variables


In [2]:
dotenv.load_dotenv()

True

## Load and Read Data


In [3]:
path = kagglehub.dataset_download("dissfya/atp-tennis-2000-2023daily-pull")
print(f"Data Path -> {path}")
dataset_dir = Path(path)
data_file = dataset_dir / "atp_tennis.csv"
df = pd.read_csv(data_file)
print(
    f"data file name -> {data_file.name}\nnumber of matches from 2000-present-> {len(df)}"
)

Data Path -> /Users/pranavrajan/.cache/kagglehub/datasets/dissfya/atp-tennis-2000-2023daily-pull/versions/961
data file name -> atp_tennis.csv
number of matches from 2000-present-> 66681


## Preprocess Data


In [4]:
# Convert to datetime objects
df["Date"] = pd.to_datetime(df["Date"])

# sort from most recent matches to earliest matches
df = df.sort_values(by="Date").reset_index(drop=True)

# # Sort by date to ensure historical features are computed correctly later
# df = df.sort_values("Date")

In [5]:
df.head()

,Tournament,Date,Series,Court,Surface,Round,Best of,Player_1,Player_2,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score
0,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Dosedel S.,Ljubicic I.,Dosedel S.,63,77,-1,-1,-1.0,-1.0,6-4 6-2
1,Qatar Open,2000-01-03,International,Outdoor,Hard,1st Round,3,El Aynaoui Y.,Dupuis A.,El Aynaoui Y.,33,78,-1,-1,-1.0,-1.0,4-6 6-3 6-2
2,Qatar Open,2000-01-03,International,Outdoor,Hard,1st Round,3,Arazi H.,Cherkasov A.,Cherkasov A.,35,206,-1,-1,-1.0,-1.0,2-6 4-6
3,Qatar Open,2000-01-03,International,Outdoor,Hard,1st Round,3,Caratti C.,Heuberger I.,Caratti C.,211,219,-1,-1,-1.0,-1.0,7-6 4-6 6-4
4,Qatar Open,2000-01-03,International,Outdoor,Hard,1st Round,3,Al-Alawi S.K.,Berasategui A.,Berasategui A.,-1,60,-1,-1,-1.0,-1.0,3-6 4-6


## Create Datasets


In [6]:
season_2025_start = pd.Timestamp("2024-12-27")
season_2025_end = pd.Timestamp("2025-11-16")

### Training Data


In [7]:
train_df = df[df["Date"] < season_2025_start].reset_index(drop=True)
print(f"Training matches (2000-2024) -> {len(train_df)}")

Training matches (2000-2024) -> 64166


In [8]:
train_df.head()

,Tournament,Date,Series,Court,Surface,Round,Best of,Player_1,Player_2,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score
0,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Dosedel S.,Ljubicic I.,Dosedel S.,63,77,-1,-1,-1.0,-1.0,6-4 6-2
1,Qatar Open,2000-01-03,International,Outdoor,Hard,1st Round,3,El Aynaoui Y.,Dupuis A.,El Aynaoui Y.,33,78,-1,-1,-1.0,-1.0,4-6 6-3 6-2
2,Qatar Open,2000-01-03,International,Outdoor,Hard,1st Round,3,Arazi H.,Cherkasov A.,Cherkasov A.,35,206,-1,-1,-1.0,-1.0,2-6 4-6
3,Qatar Open,2000-01-03,International,Outdoor,Hard,1st Round,3,Caratti C.,Heuberger I.,Caratti C.,211,219,-1,-1,-1.0,-1.0,7-6 4-6 6-4
4,Qatar Open,2000-01-03,International,Outdoor,Hard,1st Round,3,Al-Alawi S.K.,Berasategui A.,Berasategui A.,-1,60,-1,-1,-1.0,-1.0,3-6 4-6


### Testing Data


In [9]:
test_df = df[
    (df["Date"] >= season_2025_start) & (df["Date"] <= season_2025_end)
].reset_index(drop=True)
print(f"Testing matches - 2025 season -> {len(test_df)}")

Testing matches - 2025 season -> 2515


In [10]:
test_df.head()

,Tournament,Date,Series,Court,Surface,Round,Best of,Player_1,Player_2,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score
0,Brisbane International,2024-12-29,ATP250,Outdoor,Hard,1st Round,3,Vukic A.,Goffin D.,Vukic A.,68,52,778,1037,2.00,1.80,6-2 6-3
1,Hong Kong Tennis Open,2024-12-30,ATP250,Outdoor,Hard,1st Round,3,Norrie C.,Tien L.,Norrie C.,49,122,1119,493,1.67,2.20,6-3 7-5
2,Hong Kong Tennis Open,2024-12-30,ATP250,Outdoor,Hard,1st Round,3,Nakashima B.,Sonego L.,Sonego L.,38,53,1335,1026,1.57,2.38,6-7 3-6
3,Hong Kong Tennis Open,2024-12-30,ATP250,Outdoor,Hard,1st Round,3,Marozsan F.,Safiullin R.,Marozsan F.,58,60,935,923,2.63,1.50,6-1 7-6
4,Hong Kong Tennis Open,2024-12-30,ATP250,Outdoor,Hard,1st Round,3,Darderi L.,Kecmanovic M.,Kecmanovic M.,44,54,1198,1021,3.50,1.30,3-6 3-6


## Exploratory Data Analysis


### Column Types


In [11]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64166 entries, 0 to 64165
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Tournament  64166 non-null  object        
 1   Date        64166 non-null  datetime64[ns]
 2   Series      64166 non-null  object        
 3   Court       64166 non-null  object        
 4   Surface     64166 non-null  object        
 5   Round       64166 non-null  object        
 6   Best of     64166 non-null  int64         
 7   Player_1    64166 non-null  object        
 8   Player_2    64166 non-null  object        
 9   Winner      64166 non-null  object        
 10  Rank_1      64166 non-null  int64         
 11  Rank_2      64166 non-null  int64         
 12  Pts_1       64166 non-null  int64         
 13  Pts_2       64166 non-null  int64         
 14  Odd_1       64166 non-null  float64       
 15  Odd_2       64166 non-null  float64       
 16  Score       64166 non-

### Min-Max Values - Numerical Feature Columns


In [12]:
numeric_min_max = train_df.select_dtypes(include=[np.number]).agg(["min", "max"])
print(numeric_min_max)

     Best of  Rank_1  Rank_2  Pts_1  Pts_2  Odd_1  Odd_2
min        3      -1      -1     -1     -1   -1.0   -1.0
max        5    3390    4915  16950  16950   67.0   51.0


### Unique Values in Columns


In [13]:
# For each column, show unique values
for col in train_df.columns:
    unique_vals = train_df[col].unique()
    n_unique = len(unique_vals)
    print(f"\n{col} -> {n_unique} unique values")

    # Only show actual values if there are < 20 unique values
    if n_unique < 20:
        print(f"  Values: {unique_vals}")
    else:
        print("(Too many to display - showing first 10)")
        print(f"  Sample: {unique_vals[:10]}")


Tournament -> 265 unique values
(Too many to display - showing first 10)
  Sample: ['Australian Hardcourt Championships' 'Qatar Open' 'Gold Flake Open'
 'Sydney International' 'Heineken Open' 'Australian Open' 'Sybase Open'
 'Marseille Open' 'Dubai Open' 'ABN AMRO World Tennis Tournament']

Date -> 6223 unique values
(Too many to display - showing first 10)
  Sample: <DatetimeArray>
['2000-01-03 00:00:00', '2000-01-10 00:00:00', '2000-01-17 00:00:00',
 '2000-02-07 00:00:00', '2000-02-14 00:00:00', '2000-02-21 00:00:00',
 '2000-02-28 00:00:00', '2000-03-06 00:00:00', '2000-03-13 00:00:00',
 '2000-03-23 00:00:00']
Length: 10, dtype: datetime64[ns]

Series -> 8 unique values
  Values: ['International' 'Grand Slam' 'International Gold' 'Masters' 'Masters Cup'
 'ATP250' 'ATP500' 'Masters 1000']

Court -> 2 unique values
  Values: ['Outdoor' 'Indoor']

Surface -> 4 unique values
  Values: ['Hard' 'Clay' 'Grass' 'Carpet']

Round -> 8 unique values
  Values: ['1st Round' 'The Final' 'Semifina

### Number of Unique Players


In [14]:
total_unique_players = pd.concat([df["Player_1"], train_df["Player_2"]]).nunique()
print(f"Total unique players in the dataset -> {total_unique_players}")

Total unique players in the dataset -> 1755


### Total Match Stats


In [15]:
def match_stats(df):
    wins = df["Winner"].value_counts()
    p1_counts = df["Player_1"].value_counts()
    p2_counts = df["Player_2"].value_counts()
    total_matches = p1_counts.add(p2_counts, fill_value=0)
    wins = wins.reindex(total_matches.index, fill_value=0)
    losses = total_matches - wins
    player_stats = pd.DataFrame(
        {"Wins": wins, "Losses": losses, "Total Matches": total_matches}
    ).sort_values(by="Wins", ascending=False)
    print(f"len of list -> {len(player_stats)}")
    print(player_stats.head())

In [16]:
match_stats(train_df)

len of list -> 1722
             Wins  Losses  Total Matches
Federer R.   1151   242.0         1393.0
Djokovic N.  1020   191.0         1211.0
Nadal R.     1007   207.0         1214.0
Ferrer D.     677   358.0         1035.0
Murray A.     670   254.0          924.0


### Series Stats


In [17]:
def series_stats(df):
    # 1. Overall Stats: Wins, Total Matches, and Losses
    wins = df["Winner"].value_counts()
    p1_counts = df["Player_1"].value_counts()
    p2_counts = df["Player_2"].value_counts()
    total_matches = p1_counts.add(p2_counts, fill_value=0)

    # Reindex to include players with 0 wins
    wins = wins.reindex(total_matches.index, fill_value=0)
    losses = total_matches - wins

    # 2. Wins by Series
    # Unique series include 'Grand Slam', 'Masters 1000', 'ATP500', etc.
    series_wins = df.groupby(["Winner", "Series"]).size().unstack(fill_value=0)
    series_wins.columns = [f"Wins_{col}" for col in series_wins.columns]

    # 3. Create and Merge DataFrames
    player_stats = pd.DataFrame(
        {"Total Matches": total_matches, "Wins": wins, "Losses": losses}
    )

    # Join series wins to the main stats
    player_stats = player_stats.join(series_wins, how="left").fillna(0)

    # 4. Percentage Logic: Calculate series wins as a % of TOTAL wins
    win_cols = [c for c in player_stats.columns if c.startswith("Wins_")]

    # Vectorized division by total wins
    series_pcts = player_stats[win_cols].div(player_stats["Wins"], axis=0) * 100

    # Rename columns to reflect they are percentages of total wins
    series_pcts.columns = [
        c.replace("Wins_", "WinPct_Total_") for c in series_pcts.columns
    ]

    # Concatenate and handle division by zero
    player_stats = pd.concat([player_stats, series_pcts], axis=1).fillna(0)

    # Final sorting and display
    player_stats = player_stats.sort_values(by="Wins", ascending=False)

    print(f"Number of players analyzed -> {len(player_stats)}")
    print(player_stats.head())


In [18]:
series_stats(train_df)

Number of players analyzed -> 1722
             Total Matches  Wins  Losses  Wins_ATP250  Wins_ATP500  \
Federer R.          1393.0  1151   242.0         55.0        105.0   
Djokovic N.         1211.0  1020   191.0         58.0         94.0   
Nadal R.            1214.0  1007   207.0         55.0        120.0   
Ferrer D.           1035.0   677   358.0        117.0        106.0   
Murray A.            924.0   670   254.0         86.0         81.0   

             Wins_Grand Slam  Wins_International  Wins_International Gold  \
Federer R.             362.0               138.0                     62.0   
Djokovic N.            364.0                40.0                     16.0   
Nadal R.               303.0                64.0                     43.0   
Ferrer D.              139.0                80.0                     41.0   
Murray A.              195.0                57.0                     12.0   

             Wins_Masters  Wins_Masters 1000  Wins_Masters Cup  \
Federer R.     

### Court Stats


In [19]:
def court_stats(df):
    # 1. Existing Logic: Overall Wins, Total Matches, and Losses
    wins = df["Winner"].value_counts()
    p1_counts = df["Player_1"].value_counts()
    p2_counts = df["Player_2"].value_counts()
    total_matches = p1_counts.add(p2_counts, fill_value=0)

    # Reindex to include players with 0 wins
    wins = wins.reindex(total_matches.index, fill_value=0)
    losses = total_matches - wins

    # 2. New Logic: Wins by Court (Replacing Series/Surface)
    # Group by Winner and Court to get counts for 'Indoor' and 'Outdoor'
    court_wins = df.groupby(["Winner", "Court"]).size().unstack(fill_value=0)
    court_wins.columns = [f"Wins_{col}" for col in court_wins.columns]

    # 3. Create and Merge DataFrames
    player_stats = pd.DataFrame(
        {"Total Matches": total_matches, "Wins": wins, "Losses": losses}
    )

    # Join court wins to the main stats
    player_stats = player_stats.join(court_wins, how="left").fillna(0)

    # 4. Percentage Logic: Calculate court wins as a % of TOTAL wins
    win_cols = [c for c in player_stats.columns if c.startswith("Wins_")]

    # Vectorized division: Divide each court win column by the "Wins" column
    court_pcts = player_stats[win_cols].div(player_stats["Wins"], axis=0) * 100

    # Rename columns to reflect they are percentages of total career wins
    court_pcts.columns = [
        c.replace("Wins_", "WinPct_Total_") for c in court_pcts.columns
    ]

    # Concatenate and handle potential division by zero (for players with 0 wins)
    player_stats = pd.concat([player_stats, court_pcts], axis=1).fillna(0)

    # Final sorting and display
    player_stats = player_stats.sort_values(by="Wins", ascending=False)

    print(f"Number of players analyzed -> {len(player_stats)}")
    print(player_stats.head())

    # return player_stats


In [20]:
court_stats(train_df)

Number of players analyzed -> 1722
             Total Matches  Wins  Losses  Wins_Indoor  Wins_Outdoor  \
Federer R.          1393.0  1151   242.0        239.0         912.0   
Djokovic N.         1211.0  1020   191.0        146.0         874.0   
Nadal R.            1214.0  1007   207.0         76.0         931.0   
Ferrer D.           1035.0   677   358.0         90.0         587.0   
Murray A.            924.0   670   254.0        129.0         541.0   

             WinPct_Total_Indoor  WinPct_Total_Outdoor  
Federer R.             20.764553             79.235447  
Djokovic N.            14.313725             85.686275  
Nadal R.                7.547170             92.452830  
Ferrer D.              13.293944             86.706056  
Murray A.              19.253731             80.746269  


### Surface Stats


In [21]:
def surface_stats(df):
    # 1. Existing Logic: Overall Wins, Total Matches, and Losses
    wins = df["Winner"].value_counts()
    p1_counts = df["Player_1"].value_counts()
    p2_counts = df["Player_2"].value_counts()
    total_matches = p1_counts.add(p2_counts, fill_value=0)

    # Reindex to include players with 0 wins
    wins = wins.reindex(total_matches.index, fill_value=0)
    losses = total_matches - wins

    # 2. New Logic: Wins by Surface
    # Group by Winner and Surface, then count the occurrences
    surface_wins = df.groupby(["Winner", "Surface"]).size().unstack(fill_value=0)
    surface_wins.columns = [f"Wins_{col}" for col in surface_wins.columns]

    # 3. Create and Merge DataFrames
    player_stats = pd.DataFrame(
        {"Total Matches": total_matches, "Wins": wins, "Losses": losses}
    )

    # Join surface wins first so we have the columns needed for calculation
    player_stats = player_stats.join(surface_wins, how="left").fillna(0)

    # 4. NEW LOGIC: Calculate surface wins as a % of TOTAL wins
    # We select only the columns that start with "Wins_"
    win_cols = [c for c in player_stats.columns if c.startswith("Wins_")]

    # Vectorized division: Divide each surface win column by the "Wins" column
    # .div(..., axis=0) ensures it divides row-by-row
    surface_pcts = player_stats[win_cols].div(player_stats["Wins"], axis=0) * 100

    # Rename columns from "Wins_Clay" to "WinPct_Total_Clay" to clarify the denominator
    surface_pcts.columns = [
        c.replace("Wins_", "WinPct_Total_") for c in surface_pcts.columns
    ]

    # Concatenate the percentage columns and fill NaN (from 0/0 division) with 0
    player_stats = pd.concat([player_stats, surface_pcts], axis=1).fillna(0)

    # Final sorting and display
    player_stats = player_stats.sort_values(by="Wins", ascending=False)

    print(f"len of list -> {len(player_stats)}")
    print(player_stats.head())

    # return player_stats


In [22]:
surface_stats(train_df)

len of list -> 1722
             Total Matches  Wins  Losses  Wins_Carpet  Wins_Clay  Wins_Grass  \
Federer R.          1393.0  1151   242.0         46.0      210.0       182.0   
Djokovic N.         1211.0  1020   191.0          5.0      263.0       115.0   
Nadal R.            1214.0  1007   207.0          7.0      455.0        75.0   
Ferrer D.           1035.0   677   358.0          9.0      300.0        40.0   
Murray A.            924.0   670   254.0         18.0       97.0       105.0   

             Wins_Hard  WinPct_Total_Carpet  WinPct_Total_Clay  \
Federer R.       713.0             3.996525          18.245004   
Djokovic N.      637.0             0.490196          25.784314   
Nadal R.         470.0             0.695134          45.183714   
Ferrer D.        328.0             1.329394          44.313146   
Murray A.        450.0             2.686567          14.477612   

             WinPct_Total_Grass  WinPct_Total_Hard  
Federer R.            15.812337          61.94613

### Round Stats


In [23]:
def round_stats(df):
    # 1. Existing Logic: Overall Wins, Total Matches, and Losses
    wins = df["Winner"].value_counts()
    p1_counts = df["Player_1"].value_counts()
    p2_counts = df["Player_2"].value_counts()
    total_matches = p1_counts.add(p2_counts, fill_value=0)

    # Reindex to include players with 0 wins
    wins = wins.reindex(total_matches.index, fill_value=0)
    losses = total_matches - wins

    # 2. New Logic: Wins by Round (Replacing Surface)
    # Group by Winner and Round, then count the occurrences
    # Unique rounds in your data include '1st Round', 'Quarterfinals', 'The Final', etc.
    round_wins = df.groupby(["Winner", "Round"]).size().unstack(fill_value=0)
    round_wins.columns = [f"Wins_{col}" for col in round_wins.columns]

    # 3. Create and Merge DataFrames
    player_stats = pd.DataFrame(
        {"Total Matches": total_matches, "Wins": wins, "Losses": losses}
    )

    # Join round wins to the main stats
    player_stats = player_stats.join(round_wins, how="left").fillna(0)

    # 4. Percentage Logic: Calculate round wins as a % of TOTAL wins
    # Select columns that start with "Wins_" (which are now round names)
    win_cols = [c for c in player_stats.columns if c.startswith("Wins_")]

    # Vectorized division: Divide each round win column by the "Wins" column
    # Use .div(..., axis=0) to handle row-wise division
    round_pcts = player_stats[win_cols].div(player_stats["Wins"], axis=0) * 100

    # Rename columns to clarify the denominator (Total Wins)
    round_pcts.columns = [
        c.replace("Wins_", "WinPct_Total_") for c in round_pcts.columns
    ]

    # Concatenate the percentage columns and fill NaN (from 0/0 division) with 0
    player_stats = pd.concat([player_stats, round_pcts], axis=1).fillna(0)

    # Final sorting and display
    player_stats = player_stats.sort_values(by="Wins", ascending=False)

    print(f"Number of players -> {len(player_stats)}")
    print(player_stats.head())

In [24]:
round_stats(train_df)

Number of players -> 1722
             Total Matches  Wins  Losses  Wins_1st Round  Wins_2nd Round  \
Federer R.          1393.0  1151   242.0           174.0           266.0   
Djokovic N.         1211.0  1020   191.0           126.0           234.0   
Nadal R.            1214.0  1007   207.0           130.0           246.0   
Ferrer D.           1035.0   677   358.0           175.0           208.0   
Murray A.            924.0   670   254.0           144.0           183.0   

             Wins_3rd Round  Wins_4th Round  Wins_Quarterfinals  \
Federer R.            157.0            78.0               184.0   
Djokovic N.           158.0            76.0               163.0   
Nadal R.              173.0            67.0               161.0   
Ferrer D.              88.0            24.0                96.0   
Murray A.              95.0            38.0                90.0   

             Wins_Round Robin  Wins_Semifinals  Wins_The Final  \
Federer R.               41.0            149.0  

### Best of Stats


In [25]:
def best_of_stats(df):
    # 1. Overall Stats: Wins, Total Matches, and Losses
    wins = df["Winner"].value_counts()
    p1_counts = df["Player_1"].value_counts()
    p2_counts = df["Player_2"].value_counts()
    total_matches = p1_counts.add(p2_counts, fill_value=0)

    # Reindex to include players with 0 wins
    wins = wins.reindex(total_matches.index, fill_value=0)
    losses = total_matches - wins

    # 2. Wins by Match Format (Best of 3 vs 5)
    # Group by Winner and the 'Best of' column
    best_of_wins = df.groupby(["Winner", "Best of"]).size().unstack(fill_value=0)

    # Rename columns to 'Wins_3' and 'Wins_5'
    best_of_wins.columns = [f"Wins_{col}" for col in best_of_wins.columns]

    # 3. Create and Merge DataFrames
    player_stats = pd.DataFrame(
        {"Total Matches": total_matches, "Wins": wins, "Losses": losses}
    )

    # Join match format wins to the main stats
    player_stats = player_stats.join(best_of_wins, how="left").fillna(0)

    # 4. Percentage Logic: Calculate format wins as a % of TOTAL wins
    win_cols = [c for c in player_stats.columns if c.startswith("Wins_")]

    # Vectorized division by total wins
    format_pcts = player_stats[win_cols].div(player_stats["Wins"], axis=0) * 100

    # Rename columns to reflect they are percentages of total career wins
    format_pcts.columns = [
        c.replace("Wins_", "WinPct_Total_") for c in format_pcts.columns
    ]

    # Concatenate and handle potential division by zero
    player_stats = pd.concat([player_stats, format_pcts], axis=1).fillna(0)

    # Final sorting and display
    player_stats = player_stats.sort_values(by="Wins", ascending=False)

    print(f"Number of players analyzed: {len(player_stats)}")
    print(player_stats.head())

In [26]:
best_of_stats(train_df)

Number of players analyzed: 1722
             Total Matches  Wins  Losses  Wins_3  Wins_5  WinPct_Total_3  \
Federer R.          1393.0  1151   242.0   772.0   379.0       67.072111   
Djokovic N.         1211.0  1020   191.0   656.0   364.0       64.313725   
Nadal R.            1214.0  1007   207.0   697.0   310.0       69.215492   
Ferrer D.           1035.0   677   358.0   537.0   140.0       79.320532   
Murray A.            924.0   670   254.0   478.0   192.0       71.343284   

             WinPct_Total_5  
Federer R.        32.927889  
Djokovic N.       35.686275  
Nadal R.          30.784508  
Ferrer D.         20.679468  
Murray A.         28.656716  


## Data Preprocessing


### Player Statistics


In [27]:
# def preprocess_player_stats(df):
#     # 1. base statistics - wins, losses, total matches
#     wins = df["Winner"].value_counts()
#     p1_counts = df["Player_1"].value_counts()
#     p2_counts = df["Player_2"].value_counts()
#     total_matches = p1_counts.add(p2_counts, fill_value=0)
#     wins = wins.reindex(total_matches.index, fill_value=0)
#     losses = total_matches - wins

#     player_stats = pd.DataFrame(
#         {
#             "Total_Matches": total_matches,
#             "Wins": wins,
#             "Losses": losses,
#             "Win_Pct": (wins / total_matches * 100).fillna(0),
#         }
#     )

#     # 2. surface stats
#     surface_wins = df.groupby(["Winner", "Surface"]).size().unstack(fill_value=0)
#     surface_wins.columns = [f"Surface_Wins_{col}" for col in surface_wins.columns]
#     player_stats = player_stats.join(surface_wins, how="left").fillna(0)
#     surface_win_cols = [c for c in surface_wins.columns]
#     surface_pcts = (
#         player_stats[surface_win_cols].div(player_stats["Wins"], axis=0) * 100
#     )
#     surface_pcts.columns = [
#         c.replace("Wins_", "SurfaceWinPct_") for c in surface_pcts.columns
#     ]
#     player_stats = pd.concat([player_stats, surface_pcts], axis=1).fillna(0)

#     # 3. round stats
#     round_wins = df.groupby(["Winner", "Round"]).size().unstack(fill_value=0)
#     round_wins.columns = [f"Round_Wins_{col}" for col in round_wins.columns]
#     player_stats = player_stats.join(round_wins, how="left").fillna(0)
#     round_win_cols = [c for c in round_wins.columns]
#     round_pcts = player_stats[round_win_cols].div(player_stats["Wins"], axis=0) * 100
#     round_pcts.columns = [
#         c.replace("Round_Wins_", "RoundWinPct_") for c in round_pcts.columns
#     ]
#     player_stats = pd.concat([player_stats, round_pcts], axis=1).fillna(0)

#     # 4. series stats
#     series_wins = df.groupby(["Winner", "Series"]).size().unstack(fill_value=0)
#     series_wins.columns = [f"Series_Wins_{col}" for col in series_wins.columns]
#     player_stats = player_stats.join(series_wins, how="left").fillna(0)
#     series_win_cols = [c for c in series_wins.columns]
#     series_pcts = player_stats[series_win_cols].div(player_stats["Wins"], axis=0) * 100
#     series_pcts.columns = [
#         c.replace("Series_Wins_", "SeriesWinPct_") for c in series_pcts.columns
#     ]
#     player_stats = pd.concat([player_stats, series_pcts], axis=1).fillna(0)

#     # 5. court stats
#     court_wins = df.groupby(["Winner", "Court"]).size().unstack(fill_value=0)
#     court_wins.columns = [f"Court_Wins_{col}" for col in court_wins.columns]
#     player_stats = player_stats.join(court_wins, how="left").fillna(0)
#     court_win_cols = [c for c in court_wins.columns]
#     court_pcts = player_stats[court_win_cols].div(player_stats["Wins"], axis=0) * 100
#     court_pcts.columns = [
#         c.replace("Court_Wins_", "CourtWinPct_") for c in court_pcts.columns
#     ]
#     player_stats = pd.concat([player_stats, court_pcts], axis=1).fillna(0)

#     # 6. best of stats
#     best_of_wins = df.groupby(["Winner", "Best of"]).size().unstack(fill_value=0)
#     best_of_wins.columns = [f"BestOf_Wins_{int(col)}" for col in best_of_wins.columns]
#     player_stats = player_stats.join(best_of_wins, how="left").fillna(0)

#     best_of_win_cols = [c for c in best_of_wins.columns]
#     best_of_pcts = (
#         player_stats[best_of_win_cols].div(player_stats["Wins"], axis=0) * 100
#     )
#     best_of_pcts.columns = [
#         c.replace("BestOf_Wins_", "BestOfWinPct_") for c in best_of_pcts.columns
#     ]
#     player_stats = pd.concat([player_stats, best_of_pcts], axis=1).fillna(0)

#     # 7. sort by total wins
#     player_stats = player_stats.sort_values(by="Wins", ascending=False)

#     # 8. Reset index
#     player_stats = player_stats.reset_index()
#     player_stats = player_stats.rename(columns={"index": "Player_Name"})

#     return player_stats

In [28]:
# player_stats = preprocess_player_stats(train_df)

In [29]:
# player_stats.info()

In [30]:
# player_stats.head()

#### Check Preprocess Player Stats


In [31]:
# print(player_stats[player_stats["Player_Name"] == "Alcaraz C."])


In [32]:
# print(player_stats[player_stats["Player_Name"] == "Sinner J."])


#### Find Carlos Alcaraz First Match


In [33]:
# # Filter for any match where Alcaraz played as Player 1 or Player 2
# alcaraz_matches = df[
#     (df["Player_1"] == "Alcaraz C.") | (df["Player_2"] == "Alcaraz C.")
# ]
# alcaraz_first_match = alcaraz_matches.iloc[[0]]
# alcaraz_first_match


### Match Statistics


In [34]:
# def init_player_match_history():
#     return {
#         "Total_Matches": 0,
#         "Wins": 0,
#         "Losses": 0,
#         # Surface wins
#         "Wins_Hard": 0,
#         "Wins_Clay": 0,
#         "Wins_Grass": 0,
#         "Wins_Carpet": 0,
#         # Round wins
#         "Wins_1st Round": 0,
#         "Wins_2nd Round": 0,
#         "Wins_3rd Round": 0,
#         "Wins_4th Round": 0,
#         "Wins_Quarterfinals": 0,
#         "Wins_Semifinals": 0,
#         "Wins_The Final": 0,
#         "Wins_Round Robin": 0,
#         # Series wins
#         "Wins_ATP250": 0,
#         "Wins_ATP500": 0,
#         "Wins_Grand Slam": 0,
#         "Wins_Masters": 0,
#         "Wins_Masters 1000": 0,
#         "Wins_Masters Cup": 0,
#         "Wins_International": 0,
#         "Wins_International Gold": 0,
#         # Court wins
#         "Wins_Indoor": 0,
#         "Wins_Outdoor": 0,
#         # Best of wins
#         "Wins_3": 0,
#         "Wins_5": 0,
#     }


# def compute_player_match_history(df):
#     print(f"Processing {len(df)} matches chronologically...")

#     # 1. Sort chronologically
#     df = df.sort_values("Date").reset_index(drop=True)

#     # 2. Initialize player history
#     match_history = {}

#     history_rows = []

#     # Calculate median rank for missing value replacement
#     valid_ranks_1 = df[df["Rank_1"] > 0]["Rank_1"]
#     valid_ranks_2 = df[df["Rank_2"] > 0]["Rank_2"]
#     median_rank = pd.concat([valid_ranks_1, valid_ranks_2]).median()

#     print(f"  Median rank for missing values: {median_rank:.0f}")

#     # 3. Iterate through matches chronologically
#     for idx, row in df.iterrows():
#         if idx % 5000 == 0:
#             print(f"  Processed {idx}/{len(df)} matches...")

#         p1 = row["Player_1"]
#         p2 = row["Player_2"]
#         winner = row["Winner"]

#         if p1 not in match_history:
#             match_history[p1] = init_player_match_history()
#         if p2 not in match_history:
#             match_history[p2] = init_player_match_history()

#         # Handle missing values for this match
#         rank_1 = median_rank if row["Rank_1"] == -1 else row["Rank_1"]
#         rank_2 = median_rank if row["Rank_2"] == -1 else row["Rank_2"]
#         pts_1 = 0 if row["Pts_1"] == -1 else row["Pts_1"]
#         pts_2 = 0 if row["Pts_2"] == -1 else row["Pts_2"]
#         odd_1 = np.nan if row.get("Odd_1", -1) == -1 else row.get("Odd_1", -1)
#         odd_2 = np.nan if row.get("Odd_2", -1) == -1 else row.get("Odd_2", -1)

#         match_info = {
#             # Match identification
#             "Date": row["Date"],
#             "Tournament": row["Tournament"],
#             "Player_1": p1,
#             "Player_2": p2,
#             "Winner": winner,
#             # Match context (for categorical encoding later)
#             "Surface": row["Surface"],
#             "Series": row["Series"],
#             "Round": row["Round"],
#             "Court": row["Court"],
#             "Best_of": row["Best of"],
#             # Rankings (with missing value handling)
#             "Rank_1": rank_1,
#             "Rank_2": rank_2,
#             "Rank_Diff": rank_1 - rank_2,
#             # Points (with missing value handling)
#             "Pts_1": pts_1,
#             "Pts_2": pts_2,
#             "Pts_Diff": pts_1 - pts_2,
#             # Odds (with missing value handling - use NaN)
#             "Odd_1": odd_1,
#             "Odd_2": odd_2,
#             "Odds_Diff": odd_1 - odd_2 if not pd.isna(odd_1) else np.nan,
#         }

#         # Add Player 1's historical stats (before this match)
#         for stat, val in match_history[p1].items():
#             match_info[f"P1_{stat}"] = val

#         # Add Player 2's historical stats (before this match)
#         for stat, val in match_history[p2].items():
#             match_info[f"P2_{stat}"] = val

#         # Create target variable
#         match_info["Player_1_Won"] = 1 if winner == p1 else 0

#         history_rows.append(match_info)

#         # Both players played a match
#         match_history[p1]["Total_Matches"] += 1
#         match_history[p2]["Total_Matches"] += 1

#         # Update winner's stats
#         if winner == p1:
#             match_history[p1]["Wins"] += 1
#             match_history[p2]["Losses"] += 1
#             winner_state = match_history[p1]
#         else:
#             match_history[p2]["Wins"] += 1
#             match_history[p1]["Losses"] += 1
#             winner_state = match_history[p2]

#         # Update winner's context-specific wins
#         surface_key = f"Wins_{row['Surface']}"
#         round_key = f"Wins_{row['Round']}"
#         series_key = f"Wins_{row['Series']}"
#         court_key = f"Wins_{row['Court']}"
#         bestof_key = f"Wins_{row['Best of']}"

#         # Use .get() to safely handle any unexpected categories
#         winner_state[surface_key] = winner_state.get(surface_key, 0) + 1
#         winner_state[round_key] = winner_state.get(round_key, 0) + 1
#         winner_state[series_key] = winner_state.get(series_key, 0) + 1
#         winner_state[court_key] = winner_state.get(court_key, 0) + 1
#         winner_state[bestof_key] = winner_state.get(bestof_key, 0) + 1

#     print(f"Processed all {len(df)} matches")

#     # 4. Convert to DataFrame
#     atp_match_history = pd.DataFrame(history_rows)

#     print(f"match history shape -> {atp_match_history.shape}")

#     return atp_match_history


In [35]:
# complete_match_history = compute_player_match_history(train_df)

In [36]:
# complete_match_history.info()

In [37]:
# complete_match_history.head()

In [38]:
# print(f"length of train_features() -> {len(complete_match_history)}")

#### Check Alcaraz Complete Match History - First Match


In [39]:
# alcaraz_debut = complete_match_history[
#     complete_match_history["Player_1"] == "Alcaraz C."
# ].iloc[0]

# alcaraz_debut

### Calculate Player Match History Percentages


In [40]:
# def compute_match_percentages(df):
#     """
#     Convert raw win counts to percentages
#     """
#     df1 = df.copy()

#     # Overall win percentage for each player
#     df1["P1_Win_Pct"] = np.where(
#         df1["P1_Total_Matches"] > 0,
#         df1["P1_Wins"] / df1["P1_Total_Matches"],
#         0.5,  # Default for players with no history
#     )
#     df1["P2_Win_Pct"] = np.where(
#         df1["P2_Total_Matches"] > 0, df1["P2_Wins"] / df1["P2_Total_Matches"], 0.5
#     )

#     # Surface-specific win percentages
#     for surface in ["Hard", "Clay", "Grass", "Carpet"]:
#         df1[f"P1_WinPct_{surface}"] = np.where(
#             df1["P1_Total_Matches"] > 0,
#             df1[f"P1_Wins_{surface}"] / df1["P1_Total_Matches"],
#             0,
#         )
#         df1[f"P2_WinPct_{surface}"] = np.where(
#             df1["P2_Total_Matches"] > 0,
#             df1[f"P2_Wins_{surface}"] / df1["P2_Total_Matches"],
#             0,
#         )

#     # Series-specific win percentages
#     for series in ["ATP250", "ATP500", "Grand Slam", "Masters 1000"]:
#         col_name = series.replace(" ", "_")  # Handle spaces
#         df1[f"P1_WinPct_{col_name}"] = np.where(
#             df1["P1_Total_Matches"] > 0,
#             df1[f"P1_Wins_{series}"] / df1["P1_Total_Matches"],
#             0,
#         )
#         df1[f"P2_WinPct_{col_name}"] = np.where(
#             df1["P2_Total_Matches"] > 0,
#             df1[f"P2_Wins_{series}"] / df1["P2_Total_Matches"],
#             0,
#         )

#     # Court-specific win percentages
#     for court in ["Indoor", "Outdoor"]:
#         df1[f"P1_WinPct_{court}"] = np.where(
#             df1["P1_Total_Matches"] > 0,
#             df1[f"P1_Wins_{court}"] / df1["P1_Total_Matches"],
#             0,
#         )
#         df1[f"P2_WinPct_{court}"] = np.where(
#             df1["P2_Total_Matches"] > 0,
#             df1[f"P2_Wins_{court}"] / df1["P2_Total_Matches"],
#             0,
#         )

#     return df1

In [41]:
# complete_match_history_percentages = compute_match_percentages(complete_match_history)

In [42]:
# complete_match_history_percentages.info()

In [43]:
# complete_match_history_percentages.head(10)

### Categorical Feature Encoding


In [44]:
# from sklearn.preprocessing import LabelEncoder


# def encode_categorical_features(df):
#     """
#     Label encode categorical features
#     """
#     df1 = df.copy()

#     # 1. Tournament encoding (keep frequent, group rare as 'Other')
#     threshold = 0.01
#     tourney_counts = df1["Tournament"].value_counts(normalize=True)
#     df1["Tournament_Clean"] = df1["Tournament"].apply(
#         lambda x: x if tourney_counts.get(x, 0) > threshold else "Other"
#     )

#     le_tournament = LabelEncoder()
#     df1["Tournament_Encoded"] = le_tournament.fit_transform(df1["Tournament_Clean"])

#     # 2. Surface encoding
#     le_surface = LabelEncoder()
#     df1["Surface_Encoded"] = le_surface.fit_transform(df1["Surface"])

#     # 3. Series encoding
#     le_series = LabelEncoder()
#     df1["Series_Encoded"] = le_series.fit_transform(df1["Series"])

#     # 4. Round encoding
#     le_round = LabelEncoder()
#     df1["Round_Encoded"] = le_round.fit_transform(df1["Round"])

#     # 5. Court encoding
#     le_court = LabelEncoder()
#     df1["Court_Encoded"] = le_court.fit_transform(df1["Court"])

#     return df1


In [45]:
# complete_match_history_encoded = encode_categorical_features(
#     complete_match_history_percentages
# )

In [46]:
# complete_match_history_encoded.info()

In [47]:
# complete_match_history_encoded.head(10)

In [48]:
# print(f"length of categorical encoding data -> {len(complete_match_history_encoded)}")

### Derived Features


In [49]:
# def compute_derived_features(df):
#     df1 = df.copy()

#     # 1. Win percentage differential (who has better record?)
#     df1["Win_Pct_Diff"] = df1["P1_Win_Pct"] - df1["P2_Win_Pct"]

#     # 2. Experience differential (who has played more?)
#     df1["Experience_Diff"] = df1["P1_Total_Matches"] - df1["P2_Total_Matches"]

#     # 3. If playing on Hard, use Hard win % difference
#     df1["Surface_Advantage"] = 0.0

#     df1.loc[df1["Surface"] == "Hard", "Surface_Advantage"] = (
#         df1["P1_WinPct_Hard"] - df1["P2_WinPct_Hard"]
#     )
#     df1.loc[df1["Surface"] == "Clay", "Surface_Advantage"] = (
#         df1["P1_WinPct_Clay"] - df1["P2_WinPct_Clay"]
#     )
#     df1.loc[df1["Surface"] == "Grass", "Surface_Advantage"] = (
#         df1["P1_WinPct_Grass"] - df1["P2_WinPct_Grass"]
#     )
#     df1.loc[df1["Surface"] == "Carpet", "Surface_Advantage"] = (
#         df1["P1_WinPct_Carpet"] - df1["P2_WinPct_Carpet"]
#     )

#     # 4. Series advantage (Grand Slam performance difference)
#     df1.loc[df1["Series"] == "Grand Slam", "Series_Advantage"] = (
#         df1["P1_WinPct_Grand_Slam"] - df1["P2_WinPct_Grand_Slam"]
#     )
#     df1.loc[df1["Series"] == "Masters 1000", "Series_Advantage"] = (
#         df1["P1_WinPct_Masters_1000"] - df1["P2_WinPct_Masters_1000"]
#     )
#     # Add default 0 for other series
#     df1["Series_Advantage"] = df1["Series_Advantage"].fillna(0)

#     # 5. Court advantage
#     df1.loc[df1["Court"] == "Indoor", "Court_Advantage"] = (
#         df1["P1_WinPct_Indoor"] - df1["P2_WinPct_Indoor"]
#     )
#     df1.loc[df1["Court"] == "Outdoor", "Court_Advantage"] = (
#         df1["P1_WinPct_Outdoor"] - df1["P2_WinPct_Outdoor"]
#     )
#     df1["Court_Advantage"] = df1["Court_Advantage"].fillna(0)

#     return df1

In [50]:
# complete_match_history_derived_features = compute_derived_features(
#     complete_match_history_encoded
# )

In [51]:
# complete_match_history_derived_features.info()
# print(f"{len(complete_match_history_derived_features)}")

### Compute Symmetric Player Statistics


In [52]:
# def create_symmetric_dataset(df):
#     """
#     Create symmetric dataset where each match appears twice:
#     once from Player_1's perspective, once from Player_2's perspective.

#     Example:
#         Original: Federer vs Nadal (Nadal wins)
#         Creates:
#         - Row 1: P1=Federer, P2=Nadal, P1_Won=0
#         - Row 2: P1=Nadal, P2=Federer, P1_Won=1
#     """
#     print("\n=== Creating Symmetric Dataset ===")
#     print(f"Original matches: {len(df):,}")

#     df_p1_info = df.copy()

#     df_p2_info = df.copy()

#     # Swap basic match info
#     df_p2_info["Player_1"] = df["Player_2"]
#     df_p2_info["Player_2"] = df["Player_1"]

#     # Swap rankings (and invert the differential)
#     df_p2_info["Rank_1"] = df["Rank_2"]
#     df_p2_info["Rank_2"] = df["Rank_1"]
#     df_p2_info["Rank_Diff"] = -df["Rank_Diff"]

#     # Swap points (and invert the differential)
#     df_p2_info["Pts_1"] = df["Pts_2"]
#     df_p2_info["Pts_2"] = df["Pts_1"]
#     df_p2_info["Pts_Diff"] = -df["Pts_Diff"]

#     # Swap odds (and invert the differential)
#     df_p2_info["Odd_1"] = df["Odd_2"]
#     df_p2_info["Odd_2"] = df["Odd_1"]
#     df_p2_info["Odds_Diff"] = -df["Odds_Diff"]

#     # Swap ALL Player_1 and Player_2 statistics
#     # Get all P1_* and P2_* columns
#     p1_cols = [col for col in df.columns if col.startswith("P1_")]
#     p2_cols = [col for col in df.columns if col.startswith("P2_")]

#     print(f"Swapping {len(p1_cols)} P1_* columns with {len(p2_cols)} P2_* columns")

#     # Swap the columns
#     for p1_col, p2_col in zip(p1_cols, p2_cols):
#         df_p2_info[p1_col] = df[p2_col]
#         df_p2_info[p2_col] = df[p1_col]

#     diff_features = [
#         "Win_Pct_Diff",
#         "Experience_Diff",
#         "Surface_Advantage",
#         "Court_Advantage",
#     ]

#     # Add Series_Advantage if it exists
#     if "Series_Advantage" in df.columns:
#         diff_features.append("Series_Advantage")

#     for feat in diff_features:
#         if feat in df.columns:
#             df_p2_info[feat] = -df[feat]

#     df_p2_info["Player_1_Won"] = 1 - df["Player_1_Won"]

#     symmetric_df = pd.concat([df_p1_info, df_p2_info], ignore_index=True)

#     # Sort by date to maintain chronological order
#     symmetric_df = symmetric_df.sort_values("Date").reset_index(drop=True)

#     return symmetric_df


In [53]:
# complete_match_history_symmetric = create_symmetric_dataset(
#     complete_match_history_derived_features
# )


In [54]:
# complete_match_history_symmetric.info()
# print(f"Total training samples: {len(complete_match_history_symmetric):,}")


In [55]:
# complete_match_history_symmetric.head(10)

## Complete Data Preprocessing


In [56]:
from utils import (
    preprocess_player_stats,
    init_player_match_history,
    compute_player_match_history,
    compute_match_percentages,
    encode_categorical_features,
    compute_derived_features,
    create_symmetric_dataset,
)


def preprocess_data(df):
    print("Preprocess all historical data")
    complete_match_history = compute_player_match_history(df)
    complete_match_history_percentages = compute_match_percentages(
        complete_match_history
    )
    complete_encoded_match_history = encode_categorical_features(
        complete_match_history_percentages
    )
    complete_match_history_derived = compute_derived_features(
        complete_encoded_match_history
    )
    final_df = create_symmetric_dataset(complete_match_history_derived)

    return final_df


In [57]:
clean_data = preprocess_data(train_df)

Preprocess all historical data
Processing 64166 matches chronologically...
  Median rank for missing values: 54
  Processed 0/64166 matches...
  Processed 5000/64166 matches...
  Processed 10000/64166 matches...
  Processed 15000/64166 matches...
  Processed 20000/64166 matches...
  Processed 25000/64166 matches...
  Processed 30000/64166 matches...
  Processed 35000/64166 matches...
  Processed 40000/64166 matches...
  Processed 45000/64166 matches...
  Processed 50000/64166 matches...
  Processed 55000/64166 matches...
  Processed 60000/64166 matches...
Processed all 64166 matches
match history shape -> (64166, 74)

=== Creating Symmetric Dataset ===
Original matches: 64,166
Swapping 38 P1_* columns with 38 P2_* columns


In [58]:
clean_data.columns

Index(['Date', 'Tournament', 'Player_1', 'Player_2', 'Winner', 'Surface',
       'Series', 'Round', 'Court', 'Best_of',
       ...
       'Tournament_Encoded', 'Surface_Encoded', 'Series_Encoded',
       'Round_Encoded', 'Court_Encoded', 'Win_Pct_Diff', 'Experience_Diff',
       'Surface_Advantage', 'Series_Advantage', 'Court_Advantage'],
      dtype='object', length=107)

In [59]:
clean_data.head(10)

,Date,Tournament,Player_1,Player_2,Winner,Surface,Series,Round,Court,Best_of,...,Tournament_Encoded,Surface_Encoded,Series_Encoded,Round_Encoded,Court_Encoded,Win_Pct_Diff,Experience_Diff,Surface_Advantage,Series_Advantage,Court_Advantage
0,2000-01-03,Australian Hardcourt Championships,Dosedel S.,Ljubicic I.,Dosedel S.,Hard,International,1st Round,Outdoor,3,...,13,3,3,0,1,0.000000,0,0.000000,0.0,0.000000
1,2000-01-03,Gold Flake Open,Lopez-Moron A.,Vinciguerra A.,Vinciguerra A.,Hard,International,1st Round,Outdoor,3,...,13,3,3,0,1,0.500000,-1,-0.000000,-0.0,-0.000000
2,2000-01-03,Qatar Open,Gaudio G.,Luxa P.,Gaudio G.,Hard,International,1st Round,Outdoor,3,...,13,3,3,0,1,-0.000000,0,-0.000000,-0.0,-0.000000
3,2000-01-03,Qatar Open,Dupuis A.,El Aynaoui Y.,El Aynaoui Y.,Hard,International,1st Round,Outdoor,3,...,13,3,3,0,1,-0.000000,0,-0.000000,-0.0,-0.000000
4,2000-01-03,Qatar Open,Cherkasov A.,Arazi H.,Cherkasov A.,Hard,International,1st Round,Outdoor,3,...,13,3,3,0,1,-0.000000,0,-0.000000,-0.0,-0.000000
5,2000-01-03,Qatar Open,Heuberger I.,Caratti C.,Caratti C.,Hard,International,1st Round,Outdoor,3,...,13,3,3,0,1,-0.000000,0,-0.000000,-0.0,-0.000000
6,2000-01-03,Qatar Open,Berasategui A.,Al-Alawi S.K.,Berasategui A.,Hard,International,1st Round,Outdoor,3,...,13,3,3,0,1,-0.000000,0,-0.000000,-0.0,-0.000000
7,2000-01-03,Qatar Open,Novak J.,Bastl G.,Bastl G.,Hard,International,1st Round,Outdoor,3,...,13,3,3,0,1,-0.000000,0,-0.000000,-0.0,-0.000000
8,2000-01-03,Gold Flake Open,Golmard J.,Hantschk M.,Golmard J.,Hard,International,The Final,Outdoor,3,...,13,3,3,7,1,-0.000000,0,-0.000000,-0.0,-0.000000
9,2000-01-03,Gold Flake Open,Sanguinetti D.,Pioline C.,Pioline C.,Hard,International,Quarterfinals,Outdoor,3,...,13,3,3,4,1,0.333333,-1,0.333333,-0.0,0.333333


## Final Training Data


In [60]:
from utils import final_train_data

final_train_df = final_train_data(clean_data)
final_train_df["Date"] = final_train_df["Date"].dt.strftime("%Y-%m-%d")

Before: 108 columns
After: 52 columns


In [61]:
pk_cols = ["Date", "Player_1", "Player_2"]
duplicate_count = final_train_df.duplicated(subset=pk_cols).sum()
if duplicate_count > 0:
    print(f"Removing {duplicate_count} duplicates...")
    final_train_df = final_train_df.drop_duplicates(subset=pk_cols, keep="first")
    print(f"✓ Final rows: {len(final_train_df):,}")


Removing 4 duplicates...
✓ Final rows: 128,328


## Hopsworks Feature Store Setup


In [62]:
import hopsworks

project = hopsworks.login(project="ATP_Tennis_Prediction")
fs = project.get_feature_store()
tennis_matches_fg = fs.get_or_create_feature_group(
    # Name of your feature group (what you'll reference later)
    name="tennis_matches",
    # Description of what's inside
    description="ATP tennis match data (2000-2024) with player statistics, encoded categories, win percentages, and symmetric perspectives for match outcome prediction",
    # Version (start with 1, increment if you change features)
    version=1,
    primary_key=["Date", "Player_1", "Player_2"],
    # Enable for real-time predictions
    online_enabled=True,
)

2025-12-26 17:12:51,208 INFO: Initializing external client
2025-12-26 17:12:51,208 INFO: Base URL: https://eu-west.cloud.hopsworks.ai:443
2025-12-26 17:12:52,885 INFO: Python Engine initialized.

Logged in to project, explore it here https://eu-west.cloud.hopsworks.ai:443/p/3204


In [63]:
tennis_matches_fg.insert(final_train_df)
print(f"Uploaded {len(final_train_df):,} matches to Hopsworks!")

Uploading Dataframe: 100.00% |██████████| Rows 128328/128328 | Elapsed Time: 00:06 | Remaining Time: 00:00


Launching job: tennis_matches_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://eu-west.cloud.hopsworks.ai:443/p/3204/jobs/named/tennis_matches_1_offline_fg_materialization/executions
Uploaded 128,328 matches to Hopsworks!
